## Computer Vision, Assignment 1
## Craioveanu Sergiu-Ionut, 407 AI

## Data Path (CHANGE THIS)
This path is relative to the location of the notebook. Assuming the notebook is in the root folder of the project,

`train` is:
```
"train/regular_tasks/"
```

and `test` path is:

```
"test/regular_tasks/"
```

Leave the name of the variable `train_path` as is, location of given data is what actually matters. Of course, we also expect the structure given folder to match the `train` one.

If notebook fails, it's because either the versions of packages are mismatched or because path is wrong.

In [1]:
train_path = "test/regular_tasks/"

## Imports

In [2]:
import cv2 as cv
import cv2
import imutils
import os
import pickle
import numpy as np
import PIL
import skimage

from PIL import Image, ImageOps
from skimage import metrics, io
from skimage.metrics import structural_similarity as compare_ssim

```
print("Numpy version:", np.__version__)
print("OpenCV version:", cv2.__version__)
print("imutils version:", imutils.__version__)
print("Pillow version:", PIL.__version__)
print("scikit-image (skimage) version:", skimage.__version__)
```
Result:
```
Numpy version: 1.21.5
OpenCV version: 4.7.0
imutils version: 0.5.4
Pillow version: 9.2.0
scikit-image (skimage) version: 0.20.0
```

In [3]:
print("Numpy version:", np.__version__)
print("OpenCV version:", cv2.__version__)
print("imutils version:", imutils.__version__)
print("Pillow version:", PIL.__version__)
print("scikit-image (skimage) version:", skimage.__version__)

Numpy version: 1.21.5
OpenCV version: 4.7.0
imutils version: 0.5.4
Pillow version: 9.2.0
scikit-image (skimage) version: 0.20.0


## Read Features

In [4]:
# Load the template image
template = cv2.imread('template.jpg')

In [5]:
diamonds_path = "features/diamonds"
dominos_path = "features/dominos"
board_feat_path = "features/board"

In [6]:
diamonds_files = [
    f for f in os.listdir(diamonds_path) 
    if f.endswith('.jpg')
]
print(diamonds_files)

diamonds = [cv2.imread(os.path.join(diamonds_path, file)) for file in diamonds_files]
# show_image(diamonds[1])

['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg']


In [7]:
domino_files = [
    f for f in os.listdir(dominos_path) 
    if f.endswith('.jpg')
]
print(domino_files)

dominos = [cv2.imread(os.path.join(dominos_path, file)) for file in domino_files]
# show_image(dominos[1])

['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg']


In [8]:
board_files = [
    f for f in os.listdir(board_feat_path) 
    if f.endswith('.jpg')
]
print(board_files)

board_boxes = [cv2.imread(os.path.join(board_feat_path, file)) for file in board_files]
# show_image(board_boxes[1])

['0.jpg', '01.jpg', '02.jpg', '03.jpg', '04.jpg', '05.jpg', '06.jpg', '07.jpg', '08.jpg', '09.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg']


## Read Train/Test data files

In [9]:
jpg_files = [
    f for f in os.listdir(train_path) 
    if f.endswith('.jpg')
]
print(f"Image names: {jpg_files[:5]}")

txt_files = [name.split(".")[0] + ".txt" for name in jpg_files]
print(f"\nText file names: {txt_files[:5]}")

games = sorted(set([pic.split("_")[0] for pic in jpg_files]))
print(f"\nGame Indices: {games}")

games_moves = [f"{game}_moves.txt" for game in games]
print(f"\nGame Moves txt file: {games_moves}")

Image names: ['1_01.jpg', '1_02.jpg', '1_03.jpg', '1_04.jpg', '1_05.jpg']

Text file names: ['1_01.txt', '1_02.txt', '1_03.txt', '1_04.txt', '1_05.txt']

Game Indices: ['1', '2', '3', '4', '5']

Game Moves txt file: ['1_moves.txt', '2_moves.txt', '3_moves.txt', '4_moves.txt', '5_moves.txt']


## Defined Functions

### Utility

In [10]:
def show_image(input_image, window_name='image', timeout=0):
    """
    Display an image in a window, resized to 40% of its original dimensions.

    Args:
        input_image (numpy array): The input image to display.
        window_name (str, optional): The name of the window displaying the image. Defaults to 'image'.
        timeout (int, optional): The number of milliseconds to wait before closing the window. Defaults to 0.

    Note:
        If the timeout is set to 0, the window will not close automatically.
    """
    resized_image = cv.resize(
        input_image,
        (int(input_image.shape[1] * 0.4), int(input_image.shape[0] * 0.4))
    )

    cv.imshow(window_name, resized_image)
    cv.waitKey(timeout)
    cv.destroyAllWindows()


In [11]:
def plot_and_generate_template_rectangles(template_image, top_left=(35, 35, 120, 120)) -> dict:
    """
    Generate and plot template rectangles on a given image.

    Args:
        template_image (numpy array): The input image on which to draw the rectangles.
        top_left (tuple, optional): The coordinates (x, y) and dimensions (w, h) of the top-left rectangle. 
                                    Defaults to (35, 35, 120, 120).

    Returns:
        dict: A dictionary mapping row and column keys to the corresponding rectangle coordinates and dimensions.
    """
    row_keys = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15"]
    col_keys = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O"]
    keys_to_rectangle = {}

    drawing_image = template_image.copy()
    x_coord, y_coord, width, height = top_left
    for i, row in enumerate(range(x_coord, 15 * 128, 128)):
        for j, col in enumerate(range(y_coord, 15 * 126, 126)):
            cv2.rectangle(drawing_image, (col, row), (col + 120, row + 120), (0, 255, 0), 2)  # Green
            keys_to_rectangle[row_keys[i] + col_keys[j]] = (col, row, 120, 120)

    return keys_to_rectangle


keys_to_rect = plot_and_generate_template_rectangles(template)

In [12]:
def rectangle_coord_dist(a, b):
    """Given a tuple of (x, y, w, h), compare x and y"""
    return np.linalg.norm(np.array(a[:2]) - np.array(b[:2]))

In [13]:
def get_closest_box_poz(input_box: tuple) -> str:
    """
    Function that receives coordinates of a contour/rectangle, 
    and outputs the closest box coordinates.

    Args:
        input_box (tuple): The input rectangle coordinates.

    Returns:
        str: The key of the closest box in the 'keys_to_rect' dictionary.
    """
    min_distance = float('inf')
    for key, rectangle in keys_to_rect.items():
        distance = rectangle_coord_dist(rectangle, input_box)
        if distance < min_distance:
            min_distance = distance
            predicted_position = key

    return predicted_position


In [14]:
def read_frame_file(path: str):
    """FOR TESTING ONLY"""
    with open(path, 'r') as f:
        for i, line in enumerate(f):
            columns = line.split()
            if i == 0:
                coord1 = columns[0]
                num1 = columns[1]
            elif i == 1:
                coord2 = columns[0]
                num2 = columns[1]
            else:
                score = columns[0]
                
        return coord1, num1, coord2, num2, score

### Image Processing

In [15]:
def compare_move_images(input_image1, input_image2, blur_kernel_size=(9, 9), blur_std_dev=2, threshold=40):
    """
    Compare two images and find the largest contour representing the moved object.

    Args:
        input_image1 (numpy array): The first input image for comparison.
        input_image2 (numpy array): The second input image for comparison.
        blur_kernel_size (tuple, optional): The size of the Gaussian blur kernel. Defaults to (9, 9).
        blur_std_dev (int, optional): The standard deviation of the Gaussian blur kernel. Defaults to 2.
        threshold (int, optional): The threshold value for binary thresholding. Defaults to 40.

    Returns:
        numpy array: The largest contour representing the moved object.
        numpy array: The first input image with the largest contour highlighted.
        numpy array: The second input image with the largest contour highlighted.
    """
    grayscale_image1 = cv2.cvtColor(input_image1, cv2.COLOR_BGR2GRAY)
    grayscale_image2 = cv2.cvtColor(input_image2, cv2.COLOR_BGR2GRAY)

    gaussian_blur1 = cv2.GaussianBlur(grayscale_image1, blur_kernel_size, blur_std_dev)
    gaussian_blur2 = cv2.GaussianBlur(grayscale_image2, blur_kernel_size, blur_std_dev)

    abs_difference = cv2.absdiff(gaussian_blur1, gaussian_blur2)
    _, binary_difference = cv2.threshold(abs_difference, threshold, 255, cv2.THRESH_BINARY)

    morph_kernel = np.ones((5, 5), np.uint8)
    eroded_difference = cv2.erode(binary_difference, morph_kernel, iterations=1)
    contours, hierarchy = cv2.findContours(eroded_difference, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_contour = max(contours, key=cv2.contourArea)

    # Draw contours on original images
    highlighted_image1 = input_image1.copy()
    highlighted_image2 = input_image2.copy()

    (x_coord, y_coord, width, height) = cv2.boundingRect(max_contour)
    cv2.rectangle(highlighted_image1, (x_coord, y_coord), (x_coord + width, y_coord + height), (0, 0, 255), 2)
    cv2.rectangle(highlighted_image2, (x_coord, y_coord), (x_coord + width, y_coord + height), (0, 0, 255), 2)

    return max_contour, highlighted_image1, highlighted_image2


In [16]:
def split_largest_contour(largest_contour, input_image=None, display=False):
    """
    Split the largest contour in an image into two equal parts.

    Args:
        largest_contour (numpy array): The largest contour to split.
        input_image (numpy array, optional): The input image to draw the split rectangles on.
        display (bool): If True, display the drawn image and the two ROIs using the show_image() function.

    Returns:
        tuple: Two tuples containing the coordinates (x, y) and dimensions (w, h) of the split rectangles.
    """
    x_coord, y_coord, width, height = cv2.boundingRect(largest_contour)

    if width > height:
        x1, y1, w1, h1 = x_coord, y_coord, width // 2, height
        x2, y2, w2, h2 = x_coord + width // 2, y_coord, width - width // 2, height
    else:
        x1, y1, w1, h1 = x_coord, y_coord, width, height // 2
        x2, y2, w2, h2 = x_coord, y_coord + height // 2, width, height - height // 2

    if input_image is not None:
        drawn_image = input_image.copy()

        cv2.rectangle(drawn_image, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 2)  # Green
        cv2.rectangle(drawn_image, (x2, y2), (x2 + w2, y2 + h2), (0, 0, 255), 2)  # Red

        roi_left = drawn_image[y1:y1 + h1, x1:x1 + w1]
        roi_right = drawn_image[y2:y2 + h2, x2:x2 + w2]

        if display:
            show_image(drawn_image)
            show_image(roi_left)
            show_image(roi_right)

    return (x1, y1, w1, h1), (x2, y2, w2, h2)

In [17]:
def get_roi(input_image, rectangle: tuple, display=False):
    """
    Extract the region of interest (ROI) from an image based on the provided rectangle.

    Args:
        input_image (numpy array): The input image to extract the ROI from.
        rectangle (tuple): A tuple containing the coordinates (x, y) and dimensions (w, h) of the ROI.
        display (bool): If True, display the extracted ROI using the show_image() function.

    Returns:
        numpy array: The extracted region of interest (ROI) from the input image.
    """
    (x_coord, y_coord, width, height) = rectangle
    drawn_image = input_image.copy()
    cv2.rectangle(drawn_image, (x_coord, y_coord), (x_coord + width, y_coord + height), (0, 0, 255), 2)  # Red

    roi_image = input_image[y_coord:y_coord + height, x_coord:x_coord + width]
    if display:
        show_image(roi_image)

    return roi_image


In [18]:
def ssim(input_image1, input_image2, multichannel=True) -> float:
    """
    Calculate the Structural Similarity Index (SSIM) between two images.

    Args:
        input_image1 (numpy array): The first input image for comparison.
        input_image2 (numpy array): The second input image for comparison.
        multichannel (bool): If True, calculate SSIM for each channel separately and average the results.
                             If False, convert images to grayscale before calculating SSIM.

    Returns:
        float: The SSIM value between the two input images.
    """
    # Convert images to grayscale if multichannel is False
    if not multichannel:
        grayscale_image1 = cv2.cvtColor(input_image1, cv2.COLOR_BGR2GRAY)
        grayscale_image2 = cv2.cvtColor(input_image2, cv2.COLOR_BGR2GRAY)
    else:
        grayscale_image1 = input_image1
        grayscale_image2 = input_image2

    resized_image2 = cv2.resize(grayscale_image2, (grayscale_image1.shape[1], grayscale_image1.shape[0]))

    # Calculate SSIM
    ssim_result = compare_ssim(grayscale_image1, resized_image2, multichannel=multichannel, channel_axis=2)

    return ssim_result

In [19]:
def detect_domino_dots(input_image) -> int:
    """
    Detect the number of domino dots in an image using the Hough Circle Transform.

    Args:
        input_image (numpy array): The input image containing the domino.

    Returns:
        int: The count of detected domino dots.
    """
    # Convert the image to grayscale
    grayscale_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale image
    gaussian_blur = cv2.GaussianBlur(grayscale_image, (9, 9), 1)

    # Apply adaptive thresholding to emphasize the dots
    adaptive_threshold = cv2.adaptiveThreshold(
        gaussian_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # Create erosion and dilation processes for more robustness
    morph_kernel = np.ones((3, 3), np.uint8)
    morph_eroded = cv2.erode(adaptive_threshold, morph_kernel, iterations=1)
    morph_dilated = cv2.dilate(morph_eroded, morph_kernel, iterations=1)

    # Detect circles using Hough Circle Transform
    detected_circles = cv2.HoughCircles(
        morph_dilated, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=15, minRadius=5, maxRadius=20
    )

    # Count the number of detected circles (domino dots)
    dot_count = 0
    if detected_circles is not None:
        detected_circles = np.round(detected_circles[0, :]).astype("int")
        dot_count = len(detected_circles)

    return dot_count

### Scores and Game Logic

In [20]:
def first_move(initial_game_image) -> (str, int, str, int):
    """
    Determine the positions and face values of the first two dominoes placed in the game.

    Args:
        initial_game_image (numpy array): The image containing the first move in the game.

    Returns:
        tuple: A tuple containing the positions and face values of the first two dominoes 
                (position1, face_value1, position2, face_value2).
    """
    start_position = "8H"
    start_rectangle = get_roi(initial_game_image, keys_to_rect[start_position])
    face_value1 = detect_domino_dots(start_rectangle)

    max_similarity = float('-inf')
    candidate_positions = ['7H', '9H', '8G', '8I']

    for position in candidate_positions:
        candidate_rectangle = get_roi(initial_game_image, keys_to_rect[position])

        for i, domino in enumerate(dominos):
            similarity_score = ssim(candidate_rectangle, domino)

            if similarity_score > max_similarity:
                max_similarity = similarity_score
                second_position = position
                face_value2 = detect_domino_dots(candidate_rectangle)

    if start_position < second_position:
        return start_position, face_value1, second_position, face_value2
    else:
        return second_position, face_value2, start_position, face_value1


## Adjust Image Perspective and Template Match -> SIFT is ROBUST

### SIFT WARPING
This cell takes about 25 minutes for 100 images on a Ryzen 5. It warps the perspective and matches the template with each image in the train/test set. Other than this cell, there are no other compute-heavy cells. The notebook should almost instantly finish running after running the cell below.

In [26]:
%%time
warped_images = []
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
h, w = template_gray.shape

# Detect and compute features in the template image
sift = cv2.xfeatures2d.SIFT_create()
kp_template, des_template = sift.detectAndCompute(template_gray, None)

# Loop through all images in the folder
for filename in jpg_files:
    # Load the input image
    img = cv2.imread(os.path.join(train_path, filename))
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect and compute features in the input image
    kp_img, des_img = sift.detectAndCompute(gray, None)

    # Match the features between the template and input images
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des_template, des_img)
    matches = sorted(matches, key=lambda x: x.distance)

    # Find the corresponding points in the template and input images
    pts_template = np.float32([kp_template[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts_img = np.float32([kp_img[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Calculate the homography matrix and apply it to warp the input image
    H, _ = cv2.findHomography(pts_img, pts_template, cv2.RANSAC, 5.0)
    warped_img = cv2.warpPerspective(img, H, (template_gray.shape[1], template_gray.shape[0]))
    
    # Add Warped Image to list
    warped_images.append(warped_img)

    # Display the result
#     show_image(warped_img, f"Warped Img: {filename}")


Wall time: 26min 25s


In [ ]:
# for warped_img in warped_images:
#     show_image(warped_img, f"Warped Img: {filename}")

### Save and load warped sift images as pkl to not run again

In [27]:
# Save the list as a pkl file
with open('warped_images_sift_test.pkl', 'wb') as f:
    pickle.dump(warped_images, f)

In [25]:
# Load the list from the pkl file
# with open('warped_images_sift_test.pkl', 'rb') as f:
#     warped_images = pickle.load(f)

# Print the loaded list
# print(warped_images[:1])

EOFError: Ran out of input

# Main Loop
### Stats for Train:

    - 1st game 100%
    - 2nd game 100%
    - 3rd game 100%
    - 4th game 100%
    - 5th game - 93% (gresim un contur devreme si se strica scorul)


#### Game 5
```
Game: 5; Img: 5_07.jpg

poz1_t: 8O; poz2_t: 9O; face1_t: 1; face2_t: 5; score_t: 3

poz1: 8N; poz2: 9N; face1: 1; face2: 7; score: 0

Player1 idx on game_seq: 6, Face under him: 6; 
Player2 idx on game_seq: 8. Face under him: 2
-----------------------------------------------------------
Game: 5; Img: 5_09.jpg

poz1_t: 14O; poz2_t: 15O; face1_t: 3; face2_t: 4; score_t: 8

poz1: 14O; poz2: 15O; face1: 3; face2: 4; score: 5

Player1 idx on game_seq: 12, Face under him: 6; 
Player2 idx on game_seq: 13. Face under him: 2
-----------------------------------------------------------
Game: 5; Img: 5_14.jpg

poz1_t: 13J; poz2_t: 14J; face1_t: 4; face2_t: 5; score_t: 7

poz1: 13J; poz2: 14J; face1: 4; face2: 5; score: 4

Player1 idx on game_seq: 26, Face under him: 6; 
Player2 idx on game_seq: 24. Face under him: 5
-----------------------------------------------------------
Game: 5; Img: 5_16.jpg

poz1_t: 11H; poz2_t: 11I; face1_t: 0; face2_t: 1; score_t: 4

poz1: 11H; poz2: 11I; face1: 0; face2: 1; score: 1

Player1 idx on game_seq: 27, Face under him: 3; 
Player2 idx on game_seq: 25. Face under him: 0
```


    


## Score Test

Game 1: 7 faces + 3 scores (90/100)
Game 2: 4 faces, 2 poz, 2 scores (90/100)
Game 3*:2 poz, 4 faces, 3 scores (91/100)
Game 4: 4 poz, 7 f, 9 sc (80/100)
Game 5: 5 poz, 6 faces, 4 scor (85/100)

## Scoring

In [28]:
diamonds_poz = {
    "1A": 5, "1D": 4, "1H": 3, "1L": 4, "1O": 5,
    "2C": 3, "2F": 4, "2J": 4, "2M": 3,
    "3B": 3, "3E": 2, "3K": 2, "3N": 3,
    "4A": 4, "4D": 3, "4F": 2, "4J": 2, "4L": 3, "4O": 4,
    "5C": 2, "5E": 1, "5G": 1, "5I": 1, "5K": 1, "5M": 2,
    "6B": 4, "6D": 2, "6F": 1, "6J": 1, "6L": 2, "6N": 4,
    "7E": 1, "7K": 1,
    "8A": 3, "8O": 3,
    "9E": 1, "9K": 1,
    "10B": 4, "10D": 2, "10F": 1, "10J": 1, "10L": 2, "10N": 4,
    "11C": 2, "11E": 1, "11G": 1, "11I": 1, "11K": 1, "11M": 2,
    "12A": 4, "12D": 3, "12F": 2, "12J": 2, "12L": 3, "12O": 4,
    "13B": 3, "13E": 2, "13K": 2, "13N": 3,
    "14C": 3, "14F": 4, "14J": 4, "14M": 3,
    "15A": 5, "15D": 4, "15H": 3, "15L": 4, "15O": 5,
}

game_seq = [
    -1, 1, 2, 3, 4, 5, 6, 0, 2, 5, 3, # 11
    4, 6, 2, 2, 0, 3, 5, 4, 1, 6, 
    2, 4, 5, 5, 0, 6, 3, 4, 2, 0, 
    1, 5, 1, 3, 4, 4, 4, 5, 0, 6,
    3, 5, 4, 1, 3, 2, 0, 0, 1, 1,
    2, 3, 6, 3, 5, 2, 1, 0, 6, 6,
    5, 2, 1, 2, 5, 0, 3, 3, 5, 0,
]

## Main Loop

In [29]:
results_dir = "results_test/"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [30]:
%%time
# Iterate through games
for current_game in games:
    
    # Extract images belonging to current game
    current_game_imgs = [f for f in jpg_files if f.startswith(f'{current_game}')]
    
    # Extract info from lines, and split into image2player dict
    with open(train_path + f'{current_game}_moves.txt', 'r') as f:
        lines = f.readlines()

    img2player = {}
    
    try:
        for line in lines:
            columns = line.split()
            img2player[columns[0]] = columns[1] 
#             print(f'Image: {columns[0]}, Player: {columns[1]}')
    except:
        pass
    
    # Re-init player scores each game
    player_poz = {"player1": 0, "player2": 0}

    # For each image within the game
    for i, img_name in enumerate(current_game_imgs):
        
        # Re-init temp scores each move
        diamond_score = 0
        face1, face2 = 0, 0
        
        
        # Extract the index within warped images based on image name
        warp_idx = jpg_files.index(img_name)

#         # ONLY FOR CHECKING: Read frame file and check
        poz1_t, face1_t, poz2_t, face2_t, score_t = read_frame_file(train_path + txt_files[warp_idx])
        
        # We have a special case if it's our first move
        if i == 0:
            poz1, face1, poz2, face2 = first_move(warped_images[warp_idx])
        else:
            # Extract placed domino position and image before/after placing it
            largest_contour, h1, h2 = compare_move_images(warped_images[warp_idx - 1], warped_images[warp_idx])

            # Split domino contour into 2 boxes 
            left_box, right_box = split_largest_contour(largest_contour, h1)

            # 1. Get closest box for both, left and right -> done only once
            poz1 = get_closest_box_poz(left_box)
            poz2 = get_closest_box_poz(right_box)

            # Check if there's a diamond in given square
            diamond_score1 = diamonds_poz[poz1] if poz1 in diamonds_poz else 0
            diamond_score2 = diamonds_poz[poz2] if poz2 in diamonds_poz else 0
            diamond_score = diamond_score1 if diamond_score1 > diamond_score2 else diamond_score2

            # After placing the domino, extract contours
            left_box, right_box = split_largest_contour(largest_contour, h2)

            left_h2 = get_roi(h2, left_box)
            right_h2 = get_roi(h2, right_box)

            # Check which domino features have been placed
            face1 = detect_domino_dots(left_h2)
            face2 = detect_domino_dots(right_h2)  

        # -- Scoring -- 
        # If you cover a diamond with a domino, you score that many points
        # If the domino is a double domino, you score double the points
        # If you are on the track (domino face) and a domino with that number is played, 
        # every player on that face gets 3 points

        # Compute diamond score
        if face1 == face2:
            diamond_score *= 2

        # If player1 is on a domino face that has been played
        if game_seq[player_poz['player1']] == face1 or game_seq[player_poz['player1']] == face2:

            # If it's currently player1's turn, increment the current score (diamond_score)
            if img2player[img_name] == "player1":
                diamond_score += 3
            else:
                player_poz["player1"] += 3

        # If player2 is on a domino face that has been played
        if game_seq[player_poz['player2']] == face1 or game_seq[player_poz['player2']] == face2:

            # If it's currently player2's turn, increment the current score (diamond_score)
            if img2player[img_name] == "player2":
                diamond_score += 3
            else:
                player_poz["player2"] += 3
        
        # Increment the current player's score based on what we've computed
        player_poz[img2player[img_name]] += diamond_score

        # ONLY FOR CHECKING: Used to detect prediction differences
        if str(poz1) != str(poz1_t) or str(poz2) != str(poz2_t) \
            or int(face1) != int(face1_t) or int(face2) != int(face2_t) \
            or int(score_t) != int(diamond_score):

            print(f"Game: {current_game}; Img: {img_name}\n")
            print(f"poz1_t: {poz1_t}; poz2_t: {poz2_t}; face1_t: {face1_t}; face2_t: {face2_t}; score_t: {score_t}\n")
            print(f"poz1: {poz1}; poz2: {poz2}; face1: {face1}; face2: {face2}; score: {diamond_score}\n")
            print(f"Player1 idx on game_seq: {player_poz['player1']}, Face under him: {game_seq[player_poz['player1']]}; ")
            print(f"Player2 idx on game_seq: {player_poz['player2']}. Face under him: {game_seq[player_poz['player2']]}")
            print("-----------------------------------------------------------")
            
        # Write results into {results_dir}/{image_name.txt} for each image
        result_str = f"{poz1} {face1}\n{poz2} {face2}\n{diamond_score}"
        
        print(img_name, txt_files[warp_idx])
        
        with open(results_dir + txt_files[warp_idx], 'w') as f:
            f.write(result_str)

1_01.jpg 1_01.txt
Game: 1; Img: 1_02.jpg

poz1_t: 6H; poz2_t: 7H; face1_t: 5; face2_t: 6; score_t: 0

poz1: 6H; poz2: 7H; face1: 3; face2: 3; score: 0

Player1 idx on game_seq: 0, Face under him: -1; 
Player2 idx on game_seq: 0. Face under him: -1
-----------------------------------------------------------
1_02.jpg 1_02.txt
1_03.jpg 1_03.txt
Game: 1; Img: 1_04.jpg

poz1_t: 6I; poz2_t: 6J; face1_t: 5; face2_t: 6; score_t: 1

poz1: 6I; poz2: 6J; face1: 3; face2: 4; score: 1

Player1 idx on game_seq: 1, Face under him: 1; 
Player2 idx on game_seq: 1. Face under him: 1
-----------------------------------------------------------
1_04.jpg 1_04.txt
1_05.jpg 1_05.txt
1_06.jpg 1_06.txt
1_07.jpg 1_07.txt
Game: 1; Img: 1_08.jpg

poz1_t: 5D; poz2_t: 6D; face1_t: 0; face2_t: 2; score_t: 5

poz1: 5D; poz2: 6D; face1: 0; face2: 1; score: 2

Player1 idx on game_seq: 9, Face under him: 5; 
Player2 idx on game_seq: 10. Face under him: 3
-----------------------------------------------------------
1_08.jp

Game: 4; Img: 4_19.jpg

poz1_t: 11G; poz2_t: 11H; face1_t: 2; face2_t: 1; score_t: 4

poz1: 11G; poz2: 12G; face1: 2; face2: 0; score: 1

Player1 idx on game_seq: 32, Face under him: 5; 
Player2 idx on game_seq: 40. Face under him: 6
-----------------------------------------------------------
4_19.jpg 4_19.txt
Game: 4; Img: 4_20.jpg

poz1_t: 5G; poz2_t: 5H; face1_t: 0; face2_t: 5; score_t: 4

poz1: 5G; poz2: 5H; face1: 0; face2: 5; score: 1

Player1 idx on game_seq: 35, Face under him: 4; 
Player2 idx on game_seq: 41. Face under him: 3
-----------------------------------------------------------
4_20.jpg 4_20.txt
5_01.jpg 5_01.txt
Game: 5; Img: 5_02.jpg

poz1_t: 8I; poz2_t: 8J; face1_t: 2; face2_t: 1; score_t: 0

poz1: 8I; poz2: 8I; face1: 1; face2: 1; score: 0

Player1 idx on game_seq: 0, Face under him: -1; 
Player2 idx on game_seq: 0. Face under him: -1
-----------------------------------------------------------
5_02.jpg 5_02.txt
5_03.jpg 5_03.txt
5_04.jpg 5_04.txt
5_05.jpg 5_05.txt


## End of Notebook